# Explicação dos códigos utilizados para a criação da Rede Neural 

## Explicação do Código – Pré-processamento da Planilha de Dados

O trecho de código a seguir foi desenvolvido com a linguagem **Python**, utilizando as bibliotecas **Pandas** e **NumPy**, com o objetivo de realizar o pré-processamento dos dados contidos em uma planilha `.xlsx`, preparando-os para a etapa de modelagem preditiva.

```python
import pandas as pd
import numpy as np

file_path = "planilha2.xlsx"
data = pd.read_excel(file_path)
```

## Transformação de colunas categóricas em valores numéricos

```python
data['Ploidia'] = data['Ploidia'].replace({'Euplóide': 0, 'Aneuplóide': 1})
```

A variável alvo Ploidia, originalmente categórica (com valores 'Euploide' e 'Aneuplóide'), é convertida para valores binários (0 e 1, respectivamente), facilitando sua interpretação por modelos de aprendizado de máquina.

```python
data['Estágio'] = data['Estágio'].str.replace("D", "", regex=False)
data['Estágio'] = pd.to_numeric(data['Estágio'], errors='coerce')
```

A coluna Estágio, que representa o dia de desenvolvimento embrionário, possui um caractere D (ex.: "D5"). Este caractere é removido, e o valor restante é convertido para tipo numérico (inteiro), garantindo compatibilidade com algoritmos que exigem dados quantitativos.

In [ ]:
import pandas as pd
import numpy as np

file_path = "planilha2.xlsx"

data = pd.read_excel(file_path)

# Substituindo os valores da coluna 'Ploidia' de 'euploide' e 'aneuploide' para 0 e 1
data['Ploidia'] = data['Ploidia'].replace({'Euplóide': 0, 'Aneuplóide': 1})

# Removendo a letra D da coluna Estágio para ficar somente números representado o dia
data['Estágio'] = data['Estágio'].str.replace("D", "", regex=False)
data['Estágio'] = pd.to_numeric(data['Estágio'], errors='coerce')

# Substituindo a classificação Morfo pelo seu grupo de 1 a 4
def classificar_morfo(valor):
    prefixo = valor[0]
    sufixo = valor[1:]

    if sufixo == "AA" and prefixo in "3456":
        return 1  # Excelente
    elif sufixo in ["AB", "BA"] and prefixo in "3456":
        return 2  # Bom
    elif sufixo in ["BB", "AC", "CA"] and prefixo in "3456":
        return 3  # Médio
    else:
        return 4  # Ruim

# Aplicar a função diretamente na coluna 'Morfo'
data['Morfo'] = data['Morfo'].apply(classificar_morfo)

# Selecionando apenas as colunas numéricas
numeric_columns = data.select_dtypes(include=[np.number])

# Salvando em uma nova planilha
output_file = "PlanilhaNumerica2.xlsx"
numeric_columns.to_excel(output_file, index=False)

print(f"Planilha somente com dados numéricos salva como: {output_file}")

---------------------------------------------------------------------------------------------------------------

## 🧠 Explicação do Código – Avaliação do Modelo MLP e Geração de Predições

Este script em Python tem como objetivo **carregar um modelo de Rede Neural Multilayer Perceptron (MLP)** previamente treinado, aplicar esse modelo a novos dados e calcular as principais **métricas de desempenho**, além de gerar **visualizações e salvar os resultados finais** em uma nova planilha.

---

### 📚 Bibliotecas Utilizadas

```python
import pandas as pd
import joblib
import numpy as np
from sklearn.metrics import accuracy_score, confusion_matrix, roc_auc_score, classification_report, roc_curve
import matplotlib.pyplot as plt
import seaborn as sns
```

Essas bibliotecas são usadas para:
* Manipulação de dados (pandas, numpy)
* Carregamento de modelo e scaler (joblib)
* Cálculo de métricas (sklearn.metrics)
* Visualização gráfica (matplotlib, seaborn)

In [ ]:
import pandas as pd
import joblib
import numpy as np
from sklearn.metrics import accuracy_score, confusion_matrix, roc_auc_score, classification_report, roc_curve
import matplotlib.pyplot as plt
import seaborn as sns

# === 1. Carrega os dados ===
df = pd.read_excel("PlanilhaNumerica2.xlsx")

# === 2. Carrega o modelo e o scaler treinados ===
modelo = joblib.load("melhor_modelo_mlp_20250610_113659.pkl")
scaler = joblib.load("scaler_mlp_20250610_113659.pkl")

# === 3. Prepara os dados ===
X = df.drop(columns=["Ploidia"], errors="ignore")
y_true = df["Ploidia"]

# Verifica e preenche colunas faltantes
colunas_treinadas = scaler.feature_names_in_
for col in colunas_treinadas:
    if col not in X.columns:
        X[col] = 0

# Garante a ordem correta
X = X[colunas_treinadas]

# === 4. Aplica o scaler ===
X_scaled = scaler.transform(X)

# === 5. Faz a classificação binária ===
classes_preditas = modelo.predict(X_scaled)

# === 6. Obtém a probabilidade da classe positiva (euploide) ===
probabilidades = modelo.predict_proba(X_scaled)[:, 1]
porcentagem_euploidia = (probabilidades * 100).round(2)

# === 7. Adiciona as predições ao dataframe ===
df["Classe_Prevista"] = classes_preditas
df["Porcentagem_Euploidia_Preditiva"] = porcentagem_euploidia

# === 8. MÉTRICAS DE DESEMPENHO ===
acc = accuracy_score(y_true, classes_preditas)
auc = roc_auc_score(y_true, probabilidades)

cm = confusion_matrix(y_true, classes_preditas)
tn, fp, fn, tp = cm.ravel()

recall_euploide = tp / (tp + fn) if (tp + fn) > 0 else 0
recall_aneuploide = tn / (tn + fp) if (tn + fp) > 0 else 0

print("\n=== MÉTRICAS DE DESEMPENHO ===")
print(f"Acurácia: {acc:.3f}")
print(f"AUC: {auc:.3f}")
print(f"Recall Euploide (Sensibilidade): {recall_euploide:.3f}")
print(f"Recall Aneuploide (Especificidade): {recall_aneuploide:.3f}")

print("\n=== Classification Report ===")
print(classification_report(y_true, classes_preditas))

# === 9. Matriz de Confusão ===
plt.figure(figsize=(5, 4))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues",
            xticklabels=["Aneuploide (0)", "Euploide (1)"],
            yticklabels=["Aneuploide (0)", "Euploide (1)"])
plt.xlabel("Predito")
plt.ylabel("Real")
plt.title("Matriz de Confusão")
plt.tight_layout()
plt.show()

# === 10. Curva ROC ===
fpr, tpr, thresholds = roc_curve(y_true, probabilidades)
plt.figure()
plt.plot(fpr, tpr, label=f"ROC Curve (AUC = {auc:.2f})")
plt.plot([0, 1], [0, 1], linestyle='--', color='gray')
plt.xlabel("Falso Positivo (1 - Especificidade)")
plt.ylabel("Verdadeiro Positivo (Sensibilidade)")
plt.title("Curva ROC")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

# === 11. Salva a planilha final com tudo ===
df.to_excel("Planilha_Com_Classificacao_e_Porcentagem.xlsx", index=False)

print("\nConcluído com sucesso!")
print("Arquivo salvo como: Planilha_Com_Classificacao_e_Porcentagem.xlsx")

---------------------------------------------------------------------------------------------------------------

## Explicação da Integração do LIME no Pipeline de Predição

Inicialmente, foi desenvolvido um pipeline tradicional de predição utilizando uma Rede Neural Artificial do tipo Multilayer Perceptron (MLP), treinada com dados morfocinéticos de embriões. Esse pipeline inclui as etapas de pré-processamento, normalização dos dados com StandardScaler, carregamento do modelo treinado (joblib), predição das classes (euploide ou aneuploide), cálculo da probabilidade da classe positiva (euploidia) por meio do método predict_proba, e geração de métricas de desempenho como acurácia, AUC, sensibilidade (recall) e matriz de confusão. Os resultados, incluindo a classe prevista e a porcentagem de euploidia, são salvos em uma planilha Excel para posterior análise.

No entanto, apesar de fornecer predições quantitativas precisas, esse modelo opera como uma "caixa-preta", sem transparência sobre os fatores que influenciam suas decisões. Diante dessa limitação, foi incorporada ao pipeline a biblioteca LIME (Local Interpretable Model-Agnostic Explanations), com o objetivo de fornecer explicações locais para cada predição realizada.

A abordagem com LIME utiliza o LimeTabularExplainer, que gera interpretações individualizadas ao perturbar levemente os dados de entrada e observar como o modelo responde a essas variações. Dessa forma, o LIME aproxima o comportamento do modelo em torno de uma instância específica utilizando um modelo linear local, revelando quais atributos mais contribuíram para a classificação de cada embrião.

No código final, para cada embrião avaliado, o LIME é executado por meio do método explain_instance, fornecendo não apenas a probabilidade de euploidia, mas também a estrutura explicativa associada à predição. Embora neste estágio as explicações tenham sido computadas, elas ainda não foram exportadas em formato visual (por exemplo, .html), o que está previsto como uma etapa adicional para facilitar a análise interpretativa dos resultados.

A integração do LIME representa um passo fundamental na direção da inteligência artificial explicável (XAI), permitindo que os profissionais envolvidos em processos de decisão clínica compreendam melhor o raciocínio do modelo. Essa transparência é especialmente relevante no contexto da fertilização in vitro, onde decisões baseadas em aprendizado de máquina podem impactar diretamente a seleção embrionária e, consequentemente, os desfechos reprodutivos.

In [ ]:
import pandas as pd
import joblib
from sklearn.metrics import accuracy_score, confusion_matrix, roc_auc_score, classification_report, roc_curve
import matplotlib.pyplot as plt
import seaborn as sns
import lime
import lime.lime_tabular
import numpy as np

df = pd.read_excel("PlanilhaNumerica.xlsx")
modelo = joblib.load("melhor_modelo_mlp_20250610_113659.pkl")
scaler = joblib.load("scaler_mlp_20250610_113659.pkl")

X = df.drop(columns=["Ploidia"], errors="ignore")
y_true = df["Ploidia"]

colunas_treinadas = scaler.feature_names_in_
for col in colunas_treinadas:
    if col not in X.columns:
        X[col] = 0  

X = X[colunas_treinadas]
X_scaled = scaler.transform(X)

classes_preditas = modelo.predict(X_scaled)

explainer = lime.lime_tabular.LimeTabularExplainer(
    training_data=X_scaled,
    feature_names=colunas_treinadas,
    mode='classification',
    discretize_continuous=True,
    random_state=42
)

print("🔎 Calculando LIME + Probabilidades para cada embrião...")
prob_euploidia = []

for i in range(len(X_scaled)):
    _ = explainer.explain_instance(X_scaled[i], modelo.predict_proba, num_features=len(colunas_treinadas))
    
    prob = modelo.predict_proba(X_scaled[i].reshape(1, -1))[0][1]
    prob_euploidia.append(round(prob * 100, 2))

df["Classe_Prevista"] = classes_preditas
df["Prob_Euploidia_LIME"] = prob_euploidia

prob_raw = modelo.predict_proba(X_scaled)[:, 1]
acc = accuracy_score(y_true, classes_preditas)
auc = roc_auc_score(y_true, prob_raw)

cm = confusion_matrix(y_true, classes_preditas)
tn, fp, fn, tp = cm.ravel()
recall_euploide = tp / (tp + fn) if (tp + fn) > 0 else 0
recall_aneuploide = tn / (tn + fp) if (tn + fp) > 0 else 0

print("\n=== MÉTRICAS DE DESEMPENHO ===")
print(f"Acurácia: {acc:.3f}")
print(f"AUC (baseado na sigmoid): {auc:.3f}")
print(f"Recall Euploide (Sensibilidade): {recall_euploide:.3f}")
print(f"Recall Aneuploide (Especificidade): {recall_aneuploide:.3f}")
print("\n=== Classification Report ===")
print(classification_report(y_true, classes_preditas))

plt.figure(figsize=(5, 4))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues",
            xticklabels=["Aneuploide (0)", "Euploide (1)"],
            yticklabels=["Aneuploide (0)", "Euploide (1)"])
plt.xlabel("Predito")
plt.ylabel("Real")
plt.title("Matriz de Confusão")
plt.tight_layout()
plt.show()

fpr, tpr, thresholds = roc_curve(y_true, prob_raw)
plt.figure()
plt.plot(fpr, tpr, label=f"ROC Curve (AUC = {auc:.2f})")
plt.plot([0, 1], [0, 1], linestyle='--', color='gray')
plt.xlabel("Falso Positivo (1 - Especificidade)")
plt.ylabel("Verdadeiro Positivo (Sensibilidade)")
plt.title("Curva ROC")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

df.to_excel("Planilha_Com_LIME_Porcentagem.xlsx", index=False)

print("\n Concluído com sucesso!")
print("Arquivo salvo como: Planilha_Com_LIME_Porcentagem.xlsx")